In [1]:
# This file stores the filament related SQL database functions and generating QR code

In [1]:
import qrcode
import cv2
import urllib
import numpy as np
import psycopg2
import datetime

In [2]:
connection = psycopg2.connect(database='developer', user='postgres', password='4321', host='localhost')

In [3]:
cursor=connection.cursor()

In [4]:
from database import CursorFromConnectionPool
from database import Database

In [5]:
# no filamentID required for input, generate automatically when input new filament info
class Filament:
    def __init__(self, material, colour, current_loc, price, brand, left_weight):
        self.material = material
        self.colour = colour
        self.current_loc = current_loc
        self.price = price
        self.brand = brand
        self.left_weight = left_weight
    @classmethod
    # add a new roll of filament to database
    def add_new_filament(cls, material, colour, current_loc, price, brand, left_weight): 
        with CursorFromConnectionPool() as cursor:
            cursor.execute('INSERT INTO filament (material, colour, current_loc, price, brand, left_weight) VALUES (%s, %s, %s, %s, %s, %s)',
                            (material, colour, current_loc, price, brand, left_weight,))
    @classmethod
    # when take a filament off from the printer and update its weight to database
    def update_filament_weight(cls, filamentid, new_weight):
        with CursorFromConnectionPool() as cursor:
            cursor.execute('UPDATE filament SET left_weight=%s WHERE filamentid=%s;', (new_weight, filamentid,))
    
#----------------------under construction from here downwards--------------------------
    @classmethod
    # when take a filament off from the printer and update its weight to database
    def update_filament_loc(cls, filamentid, new_loc):
        with CursorFromConnectionPool() as cursor:
            cursor.execute('UPDATE filament SET current_loc=%s WHERE filamentid=%s;', (new_loc, filamentid,))

In [6]:
Database.initialise(database="developer", user="postgres", password="4321", host="localhost")

In [7]:
# no filamentID required for input, generate automatically when input new filament info
class Printer:
    def __init__(self, status, printer_model, filamentid):
        self.status = status
        self.printer_model = printer_model
        self.filamentid = filamentid
    @classmethod
    # add a new roll of filament to database
    def unload_filament_by_printer(cls, printerid): 
        with CursorFromConnectionPool() as cursor:
            cursor.execute('UPDATE printer SET filamentid=NULL WHERE printerid=%s;', (printerid,))
    @classmethod
    # when take a filament off from the printer and update its weight to database
    def load_filamentid_by_printer(cls, printerid, filamentid):
        with CursorFromConnectionPool() as cursor:
            cursor.execute('UPDATE printer SET filamentid=%s WHERE printerid=%s;', (filamentid, printerid,))
    @classmethod
    # when take a filament off from the printer and update its weight to database
    def update_status_by_printer(cls, printerid, new_status):
        with CursorFromConnectionPool() as cursor:
            cursor.execute('UPDATE printer SET status=%s WHERE printerid=%s;', (new_status, printerid,))
#----------------------under construction from here downwards--------------------------


In [9]:
Printer.update_status_by_printer('3','working')

In [2]:
# generate and save a QRcode by input filamentID
def gen_QRcode(filamentid):
    img = qrcode.make(filamentid)
    type(img)
    img.save('qr'+filamentid+'.png')

In [5]:
gen_QRcode('7')

In [26]:
# read QR code by img
def read_QRcode(img):
    try:
        detect = cv2.QRCodeDetector()
        value, points, straight_qrcode = detect.detectAndDecode(img)
        return value
    except:
        return

In [27]:
# read QR code from a octopi camera
# Keep reading until detect QRcode
def scan_QRcode_by_printer(printerid):
    W = True
    while W:
        req = urllib.request.urlopen("http://octopi"+str(printerid)+":8080/?action=snapshot")
        arr = np.asarray(bytearray(req.read()), dtype=np.uint8)
        img = cv2.imdecode(arr, -1)
        result = read_QRcode(img)
        if result != '':
            W = False
            return result

In [34]:
scan_QRcode_by_printer(3)

'2'